# Data Load

In [1]:
import config
from pshmodule.utils import filemanager as fm

In [2]:
df = fm.load(config.temp_data)

extension : .pickle
Loaded 9150 records from /Volumes/GoogleDrive/내 드라이브/MemeProject/data/temp_for_doc2vec.pickle


In [3]:
df.head()

1,content,pos,pos_name
0,나 중간고사 반에서 1등했어,"[Noun, Noun, Noun]","[나, 중간고사, 반]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[Noun, Noun, Noun, Noun, Noun, Verb, Verb]","[중간고사, 점수, 내, 반, 제일, 잘, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[Noun, Noun, Noun, Noun, Noun, Adjective]","[나, 반, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[Noun, Noun, Noun, Noun, Noun, Verb, Verb]","[우리, 반, 내, 시험, 제일, 잘, 봤다]"
4,중간고사 반 1등 먹음,"[Noun, Noun, Noun]","[중간고사, 반, 먹음]"


In [4]:
df_ref = fm.load(config.temp_ref_data)

extension : .pickle
Loaded 180940 records from /Volumes/GoogleDrive/내 드라이브/MemeProject/data/temp_for_ref.pickle


In [5]:
df_ref.cpr.fillna('', inplace=True)
df_ref.cpo.fillna('', inplace=True)

In [6]:
df_ref.head()

,u,cpr,mm,cpo
0,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 것도 곁들인...,
1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 몫을 제일 많이 곁들인...,
2,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 대춘이 몫도 곁들인...,
3,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 배보다 배꼽이 더 큰 걸 곁들인...,
4,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 과소비를 곁들인...,


# 

# Model Load

In [7]:
from konlpy.tag import Okt
from gensim.models.doc2vec import Doc2Vec


In [8]:
jvm_path = "/Library/Java/JavaVirtualMachines/zulu-15.jdk/Contents/Home/bin/java"
okt = Okt(jvmpath=jvm_path)

In [9]:
model = Doc2Vec.load(config.doc2vec)

# 

# Memebot with doc2vec Similarity

### 학습된 문장 유사도

In [10]:
index = 4
print(df.content.iloc[index])
print("-"*100)
result = model.docvecs.most_similar(index)
for k, v in enumerate(result):
    if k == 0:
        print(f"k : {k}")
        print(f"v : {v}")
        print(df.content.iloc[int(v[0])])

중간고사 반 1등 먹음
----------------------------------------------------------------------------------------------------
k : 0
v : ('4110', 0.9789978861808777)
우리 반 계주 선수로 뽑혔어


/tmp/ipykernel_42234/2948075417.py:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  result = model.docvecs.most_similar(index)


### 

### 입력 문장 유사도

In [30]:
input_sent = input()

오늘 날씨가 너무 좋다


In [31]:
# vectorize new sentences based on doc2vec
input_vec = model.infer_vector(okt.morphs(input_sent))

# most similarity top 5
return_docs = model.docvecs.most_similar(positive=[input_vec], topn=5)

# find Similar Sentences in the Trained Dataframe
result = [df.content.iloc[int(i[0])] for i in return_docs]

# result
if temp_ref.empty:
    print("유사 문장이 없습니다.")
else:
    print(f"입력 문장 : {input_sent}")
    print("-"*100)
    for k, i in enumerate(zip(return_docs, result)):
        print(f"{k+1}.")
        print(f"유사 문장 : {i[1]}")
        print(f"유사 문장 : {i[0][1]}")
        # find Similar Sentences in Dialogueset
        temp_ref = df_ref[df_ref.u == i[1]]
        temp_ref = temp_ref.sample(frac=1).reset_index(drop=True)
        result = str(temp_ref.cpr.iloc[0]) + " " + str(temp_ref.mm.iloc[0]) + " " + str(temp_ref.cpo.iloc[0])
        print(f"밈 답변 : {result.strip()}")
        print("-"*100)

입력 문장 : 오늘 날씨가 너무 좋다
----------------------------------------------------------------------------------------------------
1.
유사 문장 : 오늘 날씨가 너무 쾌청해!!
유사 문장 : 0.8447494506835938
밈 답변 : 내 덕분인 거 아니? 네가 말로만 듣던 걔가 나야↗ 날.씨.요.정
----------------------------------------------------------------------------------------------------
2.
유사 문장 : 오늘 날씨가 진짜 너무 좋은데!
유사 문장 : 0.7871317267417908
밈 답변 : 네가 말로만 듣던 걔가 나야↗ 날씨요정♪
----------------------------------------------------------------------------------------------------
3.
유사 문장 : 날씨, 오늘 대박 좋아!
유사 문장 : 0.6946655511856079
밈 답변 : 근데 피크닉... 외않나가...?
----------------------------------------------------------------------------------------------------
4.
유사 문장 : 오늘은 날씨 레알 대박이얌!
유사 문장 : 0.6709630489349365
밈 답변 : 외출하기 딱 좋은데! 외않헤...!
----------------------------------------------------------------------------------------------------
5.
유사 문장 : 오늘 날씨가 진짜 대박이다!
유사 문장 : 0.6346403956413269
밈 답변 : 후후 그거 다 내 덕분임 네가 말로만 듣던 걔가 나야↗ 날씨요정~
---------------------------------

/tmp/ipykernel_42234/2204262588.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  return_docs = model.docvecs.most_similar(positive=[input_vec], topn=5)
